<a href="https://colab.research.google.com/github/kusalsai/Ayinala_INFO5731_Spring2025/blob/main/info5731fnl_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
pip install llama-index


In [34]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import NodeParser # Import NodeParser
import tiktoken

In [35]:
!pip install --upgrade python-pptx

In [40]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceWindowNodeParser

# Assuming the PDF is directly in /content/
# If it's in a subfolder, adjust the path accordingly
document = SimpleDirectoryReader(
    input_files=["/content/[Thien Huu Nguyen][DEEP LEARNING FOR INFORMATION EXTRACTION]-1.pdf"]
).load_data()



In [63]:
from llama_index.core import Settings
# initialize settings (set chunk size)
Settings.chunk_size = 1024  # You can set this to any value you want

In [68]:
document[1]

Document(id_='a59d7e30-46ba-4358-8aaf-de627d8f6fd2', embedding=None, metadata={'page_label': 'ii', 'file_name': '[Thien Huu Nguyen][DEEP LEARNING FOR INFORMATION EXTRACTION]-1.pdf', 'file_path': '/content/[Thien Huu Nguyen][DEEP LEARNING FOR INFORMATION EXTRACTION]-1.pdf', 'file_type': 'application/pdf', 'file_size': 7952475, 'creation_date': '2025-03-19', 'last_modified_date': '2025-03-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='© Thien Huu Nguyen\nAll Rights Reserved, 2017', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}')

In [64]:
for i in document:
  print(i)
  print("/n")

Doc ID: 83e46c27-50d5-4a32-993c-f9f21d9be537
Text: DEEP LEARNING FOR INFORMATION EXTRACTION by Thien Huu Nguyen A
dissertation submitted in partial fulfillment of the requirements for
the degree of Doctor of Philosophy Department of Computer Science New
York University May, 2017 Professor Ralph Grishman
/n
Doc ID: a59d7e30-46ba-4358-8aaf-de627d8f6fd2
Text: © Thien Huu Nguyen All Rights Reserved, 2017
/n
Doc ID: 0f590758-3e07-4c6a-b854-dcac999a4c27
Text: Dedication To my beloved mother iii
/n
Doc ID: ca40a394-7ca0-48d1-ba39-e259b57f2482
Text: Acknowledgments PeoplehavedifferentstoriestotellintheirPhDtime.
MyPhDjourneybeganat a day of Fall 2012. I was supposed to meet my
advisor, Professor Ralph Grishman, for the first time in his office.
However, on my road to his office that day, I accidentally ran into
him in the Broadway avenue. It was the first time I talked to Ralph in
person af...
/n
Doc ID: 2c6038e2-8463-471b-ab7f-74645927dff2
Text: ACKNOWLEDGMENTS I own special thanks to Profess

In [94]:
def create_chunks(text, chunk_size=512):
    """
    Creates chunks of text with approximately chunk_size tokens using tiktoken.
    """
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")  # Or your desired model
    tokens = encoding.encode(text)
    chunks = []
    current_chunk = []
    current_chunk_len = 0
    for token in tokens:
        current_chunk.append(token)
        current_chunk_len += 1
        if current_chunk_len >= chunk_size:
            chunks.append(encoding.decode(current_chunk))
            current_chunk = []
            current_chunk_len = 0
    if current_chunk:  # Add any remaining tokens
        chunks.append(encoding.decode(current_chunk))
    return chunks

# Custom NodeParser to create chunks of desired size
from llama_index.core.node_parser import NodeParser
from llama_index.core.node_parser.node_utils import build_nodes_from_splits
from pydantic import Field # Import Field for custom attribute
from llama_index.core.schema import TextNode

class CustomNodeParser(NodeParser):
    chunk_size: int = Field(default=512, description="Chunk size in tokens") # Define chunk_size with Field

    def __init__(self, **kwargs):
        super().__init__(**kwargs) # Pass kwargs to superclass

    def get_nodes_from_documents(self, documents):
        """
        Creates nodes from documents, splitting text into chunks of desired size.
        """
        nodes = []
        for doc in documents:
            chunks = create_chunks(doc.text, self.chunk_size)
            # Fix: Use build_nodes_from_splits for proper node creation
            # Instead of passing doc_id directly, set it after node creation
            for i, chunk in enumerate(chunks):
                node = TextNode(text=chunk, id_=f"{doc.id_}-{i}")
                nodes.append(node)
        return nodes

    # Implement the _parse_nodes method
    def _parse_nodes(self, text):
        """This is a placeholder for custom parsing logic.
        It will not split text by default."""
        return [text]

# Create an instance of the custom NodeParser
node_parser = CustomNodeParser(chunk_size=1024)

# Create semantic chunks
chunks = node_parser.get_nodes_from_documents(document)

# Print the first few chunks (for demonstration)
for i, chunk in enumerate(chunks[:10]):
    print(f"Chunk {i+1}:\n{chunk.text}\n")

Chunk 1:
DEEP LEARNING FOR INFORMATION EXTRACTION
by
Thien Huu Nguyen
A dissertation submitted in partial fulfillment
of the requirements for the degree of
Doctor of Philosophy
Department of Computer Science
New York University
May, 2017
Professor Ralph Grishman

Chunk 2:
© Thien Huu Nguyen
All Rights Reserved, 2017

Chunk 3:
Dedication
To my beloved mother
iii

Chunk 4:
Acknowledgments
PeoplehavedifferentstoriestotellintheirPhDtime. MyPhDjourneybeganat
a day of Fall 2012. I was supposed to meet my advisor, Professor Ralph Grishman,
for the first time in his office. However, on my road to his office that day, I
accidentally ran into him in the Broadway avenue. It was the first time I talked to
Ralph in person after the many discussions via emails. The unexpected meeting
with Ralph impressed me so much that I forgot my nerve and worry of the first
days I entered United States to pursue a PhD degree. Ralph was so nice and
approachable that I could immediately feel the trust and confidenc

In [98]:
pip install chromadb

In [95]:
!pip uninstall llama-index-vector-stores-chroma -y

In [96]:
!pip install --upgrade llama-index

In [99]:
# Install the necessary packages
!pip install --upgrade llama-index
!pip uninstall llama-index-vector-stores-chroma -y # uninstall the conflicting package

# Import necessary libraries
from llama_index.vector_stores import ChromaVectorStore # import after upgrading llama-index

ImportError: cannot import name 'ChromaVectorStore' from 'llama_index.vector_stores' (unknown location)

In [100]:
!pip install llama-index-vector-stores-chroma # Install the necessary package to add vector_stores to llama_index
# Install the necessary packages
!pip install --upgrade llama-index
!pip uninstall llama-index-vector-stores-chroma -y # uninstall the conflicting package

# Import necessary libraries


  Using cached llama_index_vector_stores_chroma-0.4.1-py3-none-any.whl.metadata (696 bytes)
Using cached llama_index_vector_stores_chroma-0.4.1-py3-none-any.whl (5.2 kB)
Found existing installation: llama-index-vector-stores-chroma 0.4.1
Uninstalling llama-index-vector-stores-chroma-0.4.1:
  Successfully uninstalled llama-index-vector-stores-chroma-0.4.1


In [101]:
!pip install llama-index-vector-stores-chroma
from typing import Collection
from pathlib import Path
from chromadb import PersistentClient
from chromadb.api.types import Documents, Embeddings, Metadatas, IDs
from llama_index.vector_stores.chroma import ChromaVectorStore  # Correct import



  Using cached llama_index_vector_stores_chroma-0.4.1-py3-none-any.whl.metadata (696 bytes)
Using cached llama_index_vector_stores_chroma-0.4.1-py3-none-any.whl (5.2 kB)


In [102]:

chroma_db = "./chroma_db"

# Ensure the directory exists
Path(chroma_db).mkdir(parents=True, exist_ok=True)

try:
    # Initialize ChromaDB
    chroma_client = PersistentClient(path=str(chroma_db))
    vector_store = ChromaVectorStore(chroma_client, collection_name="chunks")

    print("Chroma DB installed and initialized successfully")
except Exception as e:
    print(f"Chroma DB encountered an error: {e}")

Chroma DB installed and initialized successfully


In [103]:
# Install necessary packages
!pip install --upgrade llama-index
!pip uninstall llama-index-vector-stores-chroma -y # uninstall the conflicting package



Found existing installation: llama-index-vector-stores-chroma 0.4.1
Uninstalling llama-index-vector-stores-chroma-0.4.1:
  Successfully uninstalled llama-index-vector-stores-chroma-0.4.1


In [51]:
from llama_index.core.ingestion import IngestionPipeline  # Correct import for IngestionPipeline
from llama_index.core.node_parser.text import SentenceSplitter  # Correct import for SentenceSplitter
from llama_index.core.node_parser.text import TokenTextSplitter  # Correct import for TokenTextSplitter

# Ensure you have the vector_store object properly initialized
pipeline = IngestionPipeline(
    transformations=[
        TokenTextSplitter(chunk_size=512),  # TokenTextSplitter now requires a chunk_size parameter
        SentenceSplitter(chunk_size=512, chunk_overlap=0),  # Corrected parameter name
    ],
    vector_store=vector_store,
)

print("Ingestion pipeline initialized successfully!")


Ingestion pipeline initialized successfully!


In [104]:

try:
  nodes = pipeline.run(documents=document)
  if not nodes:
    print("no nodes were created")
    exit()
  print(f"{len(nodes)} document nodes created in chroma db")
except Exception as e:
  print(f"An error occurred: {e}")


278 document nodes created in chroma db


In [105]:
try:
  nodes = pipeline.run(documents=document)
  if not nodes:
    print("no nodes were created")
    exit()
  print(f"{len(nodes)} document nodes created in chroma db")

  # Print all nodes:
  for i, node in enumerate(nodes):
    print(f"Node {i + 1}:")
    print(f"  Text: {node.text}")  # Print the text content of the node
    print(f"  ID: {node.id_}")    # Print the unique ID of the node
    print("-" * 20)  # Add a separator for better readability

except Exception as e:
  print(f"An error occurred: {e}")

278 document nodes created in chroma db
Node 1:
  Text: DEEP LEARNING FOR INFORMATION EXTRACTION
by
Thien Huu Nguyen
A dissertation submitted in partial fulfillment
of the requirements for the degree of
Doctor of Philosophy
Department of Computer Science
New York University
May, 2017
Professor Ralph Grishman
  ID: b337f773-3037-4d32-a1c9-379c5750cc50
--------------------
Node 2:
  Text: © Thien Huu Nguyen
All Rights Reserved, 2017
  ID: 5e389575-25fd-4914-869f-3b4a4978dab4
--------------------
Node 3:
  Text: Dedication
To my beloved mother
iii
  ID: 9b0314f9-b100-4cdb-a4f2-4d9c0ece7b06
--------------------
Node 4:
  Text: Acknowledgments
PeoplehavedifferentstoriestotellintheirPhDtime. MyPhDjourneybeganat
a day of Fall 2012. I was supposed to meet my advisor, Professor Ralph Grishman,
for the first time in his office. However, on my road to his office that day, I
accidentally ran into him in the Broadway avenue. It was the first time I talked to
Ralph in person after the many discussio

Task

Remove textbooks with pdfs from database

Keep only important information like ppts, ipynbs and some pdfs related to the module.

(Maybe like 3-4 important files for each module)

Then parse, make sure all nodes or chunk is of same size and maintaine good overlap between each chunk.

Metadata is important add metadata (file_path, id, some important information to easily indentity which chunk it retriving)

Then, try with some querys to find relevant information from vectordatabase.